# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2884b18c70>
├── 'a' --> tensor([[ 0.0779, -1.5959, -1.3566],
│                   [-1.2300, -2.0051,  0.9038]])
└── 'x' --> <FastTreeValue 0x7f2884b18c40>
    └── 'c' --> tensor([[-0.2267,  0.3418,  0.2282, -0.2655],
                        [-2.3643, -0.3023, -0.1733, -0.5833],
                        [ 1.7385,  1.3557,  0.6516, -0.8202]])

In [4]:
t.a

tensor([[ 0.0779, -1.5959, -1.3566],
        [-1.2300, -2.0051,  0.9038]])

In [5]:
%timeit t.a

63.8 ns ± 0.117 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2884b18c70>
├── 'a' --> tensor([[-0.9932, -0.2601, -0.1468],
│                   [ 0.0334, -1.9604,  1.9036]])
└── 'x' --> <FastTreeValue 0x7f2884b18c40>
    └── 'c' --> tensor([[-0.2267,  0.3418,  0.2282, -0.2655],
                        [-2.3643, -0.3023, -0.1733, -0.5833],
                        [ 1.7385,  1.3557,  0.6516, -0.8202]])

In [7]:
%timeit t.a = new_value

66.5 ns ± 0.0835 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0779, -1.5959, -1.3566],
               [-1.2300, -2.0051,  0.9038]]),
    x: Batch(
           c: tensor([[-0.2267,  0.3418,  0.2282, -0.2655],
                      [-2.3643, -0.3023, -0.1733, -0.5833],
                      [ 1.7385,  1.3557,  0.6516, -0.8202]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0779, -1.5959, -1.3566],
        [-1.2300, -2.0051,  0.9038]])

In [11]:
%timeit b.a

59.2 ns ± 0.0377 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4483,  0.9863, -1.8117],
               [ 0.5261,  1.6320,  1.5772]]),
    x: Batch(
           c: tensor([[-0.2267,  0.3418,  0.2282, -0.2655],
                      [-2.3643, -0.3023, -0.1733, -0.5833],
                      [ 1.7385,  1.3557,  0.6516, -0.8202]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.126 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

824 ns ± 0.357 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 744 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 405 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f27cee59a30>
├── 'a' --> tensor([[[ 0.0779, -1.5959, -1.3566],
│                    [-1.2300, -2.0051,  0.9038]],
│           
│                   [[ 0.0779, -1.5959, -1.3566],
│                    [-1.2300, -2.0051,  0.9038]],
│           
│                   [[ 0.0779, -1.5959, -1.3566],
│                    [-1.2300, -2.0051,  0.9038]],
│           
│                   [[ 0.0779, -1.5959, -1.3566],
│                    [-1.2300, -2.0051,  0.9038]],
│           
│                   [[ 0.0779, -1.5959, -1.3566],
│                    [-1.2300, -2.0051,  0.9038]],
│           
│                   [[ 0.0779, -1.5959, -1.3566],
│                    [-1.2300, -2.0051,  0.9038]],
│           
│                   [[ 0.0779, -1.5959, -1.3566],
│                    [-1.2300, -2.0051,  0.9038]],
│           
│                   [[ 0.0779, -1.5959, -1.3566],
│                    [-1.2300, -2.0051,  0.9038]]])
└── 'x' --> <FastTreeValue 0x7f27cee04f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f27cf426910>
├── 'a' --> tensor([[ 0.0779, -1.5959, -1.3566],
│                   [-1.2300, -2.0051,  0.9038],
│                   [ 0.0779, -1.5959, -1.3566],
│                   [-1.2300, -2.0051,  0.9038],
│                   [ 0.0779, -1.5959, -1.3566],
│                   [-1.2300, -2.0051,  0.9038],
│                   [ 0.0779, -1.5959, -1.3566],
│                   [-1.2300, -2.0051,  0.9038],
│                   [ 0.0779, -1.5959, -1.3566],
│                   [-1.2300, -2.0051,  0.9038],
│                   [ 0.0779, -1.5959, -1.3566],
│                   [-1.2300, -2.0051,  0.9038],
│                   [ 0.0779, -1.5959, -1.3566],
│                   [-1.2300, -2.0051,  0.9038],
│                   [ 0.0779, -1.5959, -1.3566],
│                   [-1.2300, -2.0051,  0.9038]])
└── 'x' --> <FastTreeValue 0x7f27cedec040>
    └── 'c' --> tensor([[-0.2267,  0.3418,  0.2282, -0.2655],
                        [-2.3643, -0.3023, -0.1733, -0.5833],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 68.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.8 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0779, -1.5959, -1.3566],
                [-1.2300, -2.0051,  0.9038]],
       
               [[ 0.0779, -1.5959, -1.3566],
                [-1.2300, -2.0051,  0.9038]],
       
               [[ 0.0779, -1.5959, -1.3566],
                [-1.2300, -2.0051,  0.9038]],
       
               [[ 0.0779, -1.5959, -1.3566],
                [-1.2300, -2.0051,  0.9038]],
       
               [[ 0.0779, -1.5959, -1.3566],
                [-1.2300, -2.0051,  0.9038]],
       
               [[ 0.0779, -1.5959, -1.3566],
                [-1.2300, -2.0051,  0.9038]],
       
               [[ 0.0779, -1.5959, -1.3566],
                [-1.2300, -2.0051,  0.9038]],
       
               [[ 0.0779, -1.5959, -1.3566],
                [-1.2300, -2.0051,  0.9038]]]),
    x: Batch(
           c: tensor([[[-0.2267,  0.3418,  0.2282, -0.2655],
                       [-2.3643, -0.3023, -0.1733, -0.5833],
                       [ 1.7385,  1.3557,  0.6516, -0.8202]],
         

In [26]:
%timeit Batch.stack(batches)

77.1 µs ± 180 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0779, -1.5959, -1.3566],
               [-1.2300, -2.0051,  0.9038],
               [ 0.0779, -1.5959, -1.3566],
               [-1.2300, -2.0051,  0.9038],
               [ 0.0779, -1.5959, -1.3566],
               [-1.2300, -2.0051,  0.9038],
               [ 0.0779, -1.5959, -1.3566],
               [-1.2300, -2.0051,  0.9038],
               [ 0.0779, -1.5959, -1.3566],
               [-1.2300, -2.0051,  0.9038],
               [ 0.0779, -1.5959, -1.3566],
               [-1.2300, -2.0051,  0.9038],
               [ 0.0779, -1.5959, -1.3566],
               [-1.2300, -2.0051,  0.9038],
               [ 0.0779, -1.5959, -1.3566],
               [-1.2300, -2.0051,  0.9038]]),
    x: Batch(
           c: tensor([[-0.2267,  0.3418,  0.2282, -0.2655],
                      [-2.3643, -0.3023, -0.1733, -0.5833],
                      [ 1.7385,  1.3557,  0.6516, -0.8202],
                      [-0.2267,  0.3418,  0.2282, -0.2655],
                      [-2.3643, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 433 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
